# Summarisation — T5-small on CNN/DailyMail
**TL;DR:** Summarise news articles with T5-small and validate ROUGE on a tiny CPU slice.

**Models & Datasets:** [t5-small](https://huggingface.co/t5-small) (Apache-2.0), [CNN/DailyMail 3.0.0](https://huggingface.co/datasets/cnn_dailymail) (NonCommercial)
**Run Profiles:** 🖥️ CPU | 🍎 Metal (Apple Silicon) | 🧪 Colab/T4 | ⚡ CUDA GPU
**Env (minimal):** python>=3.10, transformers, datasets, evaluate, accelerate (optional: peft, bitsandbytes, timm, diffusers)
**Colab:** [Open in Colab](https://colab.research.google.com/github/SSusantAchary/Hands-On-Huggingface-AI-Models/blob/main/notebooks/nlp/summarization-t5-small-cnn_dm_cpu-first.ipynb)

**Switches (edit in one place):**
- `device` = {"cpu","mps","cuda"}
- `precision` = {"fp32","fp16","bf16","int8","4bit"}  (apply only if supported)
- `context_len` / `image_res` / `batch_size`

**Footprint & Speed (fill after run):**
- Peak RAM: TODO
- Peak VRAM: TODO (if GPU)
- TTFB: TODO, Throughput: TODO, Load time: TODO

**Gotchas:** Long contexts can OOM tokenizers—clip inputs to 512 tokens ([Fixes & Tips](../fixes-and-tips/tokenizer-context-oom.md))



## Setup
Slice the CNN/DailyMail dataset and prepare the summarisation pipeline.


In [ ]:

import json
import os
import subprocess
import time
from pathlib import Path

import pandas as pd
import torch
from datasets import load_dataset
from evaluate import load as load_metric
from transformers import pipeline

from notebooks._templates.measure import append_benchmark_row, measure_memory_speed

DEVICE_PREFERENCE = os.environ.get("HF_DEVICE", "cpu")
PRECISION = os.environ.get("HF_PRECISION", "fp32")
MAX_INPUT_TOKENS = int(os.environ.get("HF_CONTEXT_LEN", "512"))
BATCH_SIZE = int(os.environ.get("HF_BATCH", "2"))

def resolve_device(preference: str = "cpu") -> str:
    if preference == "cuda" and torch.cuda.is_available():
        return "cuda:0"
    if preference == "mps" and torch.backends.mps.is_available():
        return "mps"
    return "cpu"

DEVICE = resolve_device(DEVICE_PREFERENCE)
print(f"Using device={DEVICE}, max_input_tokens={MAX_INPUT_TOKENS}")

MODEL_ID = "t5-small"
DATASET_ID = "cnn_dailymail"
DATASET_CONFIG = "3.0.0"
OUTPUT_DIR = Path("outputs") / "summarization-t5-small"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

dataset = load_dataset(DATASET_ID, DATASET_CONFIG, split="test[:4]")
sources = dataset["article"]
references = dataset["highlights"]


## Generate summaries


In [ ]:

torch.manual_seed(42)

load_start = time.perf_counter()
summariser = pipeline(
    "summarization",
    model=MODEL_ID,
    device=DEVICE,
    batch_size=BATCH_SIZE,
    truncation=True,
    max_length=128,
    min_length=32,
)
load_time = time.perf_counter() - load_start

generated = summariser(
    sources,
    truncation=True,
    max_length=128,
    min_length=32,
    clean_up_tokenization_spaces=True,
)
summaries = [item["summary_text"] for item in generated]

comparison = pd.DataFrame(
    {
        "source": [src[:200].replace("\n", " ") + "…" for src in sources],
        "summary": summaries,
        "reference": references,
    }
)
display(comparison)

rouge = load_metric("rouge")
rouge_scores = rouge.compute(predictions=summaries, references=references, use_stemmer=True)
print({k: round(v.mid.fmeasure, 3) for k, v in rouge_scores.items()})

out_path = OUTPUT_DIR / "summaries.csv"
comparison.to_csv(out_path, index=False)
print(f"Wrote {out_path}")


## Measurement


In [ ]:

def run_inference(recorder):
    outputs = summariser(
        sources,
        truncation=True,
        max_length=128,
        min_length=32,
    )
    if outputs:
        recorder.mark_first_token()
    recorder.add_items(len(outputs))

metrics = measure_memory_speed(run_inference)

def fmt(value, digits=4):
    if value in (None, "", float("inf")):
        return ""
    return f"{value:.{digits}f}"

try:
    repo_commit = subprocess.check_output(["git", "rev-parse", "HEAD"], text=True).strip()
except Exception:  # noqa: BLE001
    repo_commit = ""

append_benchmark_row(
    task="summarization-cnn_dm",
    model_id=MODEL_ID,
    dataset=f"{DATASET_ID}:{DATASET_CONFIG}",
    sequence_or_image_res=f"{MAX_INPUT_TOKENS}-tokens",
    batch=str(BATCH_SIZE),
    peak_ram_mb=fmt(metrics.get("peak_ram_mb"), 2),
    peak_vram_mb=fmt(metrics.get("peak_vram_mb"), 2),
    load_time_s=fmt(load_time, 2),
    ttfb_s=fmt(metrics.get("ttfb_s"), 3),
    tokens_per_s_or_images_per_s=fmt(metrics.get("throughput_per_s"), 3),
    precision=PRECISION,
    notebook_path="notebooks/nlp/summarization-t5-small-cnn_dm_cpu-first.ipynb",
    repo_commit=repo_commit,
)

with open(OUTPUT_DIR / "metrics.json", "w", encoding="utf-8") as fp:
    json.dump(metrics, fp, indent=2)
metrics


## Results Summary
        - Observations: TODO
        - Metrics captured: see `benchmarks/matrix.csv`

        ## Next Steps
        - TODOs: fill in after benchmarking

        ## Repro
        - Seed: 42 (set in measurement cell)
        - Libraries: captured via `detect_env()`
        - Notebook path: `notebooks/nlp/summarization-t5-small-cnn_dm_cpu-first.ipynb`
        - Latest commit: populated automatically when appending benchmarks (if git available)
